In [ ]:
!git clone https://github.com/pooja-premnath/CheckThat-Task2-Subjectivity

Cloning into 'CheckThat-Task2-Subjectivity'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 19 (delta 7), reused 18 (delta 6), pack-reused 0
Receiving objects: 100% (19/19), 173.30 KiB | 5.78 MiB/s, done.
Resolving deltas: 100% (7/7), done.


# Basic LSTM and BiLSTM




In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import f1_score

# Load data
df_final = pd.read_csv("/content/CheckThat-Task2-Subjectivity/train_en.tsv", sep='\t')
df_test_final = pd.read_csv("/content/CheckThat-Task2-Subjectivity/dev_test_en.tsv", sep='\t')
df_submission_final = pd.read_csv("/content/CheckThat-Task2-Subjectivity/test_en.tsv", sep='\t')

# Tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_final['sentence'])
X_train = tokenizer.texts_to_sequences(df_final['sentence'])
X_test = tokenizer.texts_to_sequences(df_test_final['sentence'])
X_submission = tokenizer.texts_to_sequences(df_submission_final['sentence'])

max_length = 100  # Choose an appropriate max length
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')
X_submission = pad_sequences(X_submission, maxlen=max_length, padding='post')

# Encode labels
label_encoder = LabelEncoder()
df_final['Encoded_Label'] = label_encoder.fit_transform(df_final['label'])
y_train = df_final['Encoded_Label']
y_test = label_encoder.transform(df_test_final['label'])

# Define RNN model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=max_length),
    LSTM(units=128),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

# Evaluate on test data
_, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy on test data: {accuracy}")

# Predict probabilities on submission data
y_submission_prob = model.predict(X_submission)

# Convert probabilities to class labels based on threshold (0.5)
y_submission_pred = (y_submission_prob > 0.5).astype(int)
y_submission_pred = label_encoder.inverse_transform(y_submission_pred.flatten())

# Replace encoded labels with original labels
df_submission_final['Predicted_Label'] = y_submission_pred

# Save submission DataFrame to TSV with sentence_id and original labels
df_submission_final[['sentence_id', 'Predicted_Label']].to_csv("submission_predictions.tsv", sep='\t', index=False)

# Calculate macro average F1 score on test set
y_test_prob = model.predict(X_test)
y_test_pred = (y_test_prob > 0.5).astype(int)
y_test_pred = y_test_pred.flatten()
macro_f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Macro Average F1 Score on test set: {macro_f1}")


Epoch 1/20
24/24 [==============================] - 9s 182ms/step - loss: 0.6581 - accuracy: 0.6439 - val_loss: 0.6669 - val_accuracy: 0.6145
Epoch 2/20
24/24 [==============================] - 5s 215ms/step - loss: 0.6529 - accuracy: 0.6439 - val_loss: 0.6725 - val_accuracy: 0.6145
Epoch 3/20
24/24 [==============================] - 4s 153ms/step - loss: 0.6552 - accuracy: 0.6439 - val_loss: 0.6667 - val_accuracy: 0.6145
Epoch 4/20
24/24 [==============================] - 4s 155ms/step - loss: 0.6539 - accuracy: 0.6439 - val_loss: 0.6684 - val_accuracy: 0.6145
Epoch 5/20
24/24 [==============================] - 5s 212ms/step - loss: 0.6521 - accuracy: 0.6439 - val_loss: 0.6690 - val_accuracy: 0.6145
Epoch 6/20
8/8 [==============================] - 11s 51ms/step - loss: 0.7283 - accuracy: 0.4905
Accuracy on test data: 0.49054276943206787
8/8 [==============================] - 0s 47ms/step
Macro Average F1 Score on test set: 0.3231197771587744


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import f1_score

# Load data
df_final = pd.read_csv("/content/CheckThat-Task2-Subjectivity/train_en.tsv", sep='\t')
df_test_final = pd.read_csv("/content/CheckThat-Task2-Subjectivity/dev_test_en.tsv", sep='\t')
df_submission_final = pd.read_csv("/content/CheckThat-Task2-Subjectivity/test_en.tsv", sep='\t')

# Tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_final['sentence'])

# Encode text data to sequences
X_train = tokenizer.texts_to_sequences(df_final['sentence'])
X_test = tokenizer.texts_to_sequences(df_test_final['sentence'])
X_submission = tokenizer.texts_to_sequences(df_submission_final['sentence'])

# Pad sequences to ensure uniform length
max_sequence_length = max([len(seq) for seq in X_train + X_test + X_submission])
X_train = pad_sequences(X_train, maxlen=max_sequence_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_sequence_length, padding='post')
X_submission = pad_sequences(X_submission, maxlen=max_sequence_length, padding='post')

# Encode labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df_final['label'])
y_test = label_encoder.transform(df_test_final['label'])

# Build highly complex LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(units=512, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(units=256, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(units=128)))
model.add(Dropout(0.5))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=30, batch_size=128, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Predict probabilities for test set
y_pred_probs = model.predict(X_test)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)

# Calculate accuracy
accuracy = (y_pred == y_test).mean()
print(f'Accuracy on Test Set: {accuracy}')

# Calculate macro average F1 score
f1 = f1_score(y_test, y_pred, average='macro')
print(f'Macro Average F1 Score on Test Set: {f1}')

# Predict probabilities for submission data
y_submission_pred_probs = model.predict(X_submission)

# Convert probabilities to class labels
y_submission_pred = np.argmax(y_submission_pred_probs, axis=1)
predicted_labels = label_encoder.inverse_transform(y_submission_pred)

# Check unique predicted labels
unique_labels = np.unique(predicted_labels)
print(f'Unique Predicted Labels: {unique_labels}')

# Create dataframe with sentence_id and predicted labels
df_submission_predicted = pd.DataFrame({'sentence_id': df_submission_final['sentence_id'], 'predicted_label': predicted_labels})

# Save dataframe to TSV file
df_submission_predicted.to_csv('submission_predictions_complex_lstm_updated.tsv', sep='\t', index=False)


Epoch 1/30
7/7 [==============================] - 126s 17s/step - loss: 0.6687 - accuracy: 0.6217 - val_loss: 0.8599 - val_accuracy: 0.4774
Epoch 2/30
7/7 [==============================] - 97s 14s/step - loss: 0.6712 - accuracy: 0.6386 - val_loss: 0.7409 - val_accuracy: 0.4774
Epoch 3/30
7/7 [==============================] - 99s 14s/step - loss: 0.6413 - accuracy: 0.6398 - val_loss: 0.6902 - val_accuracy: 0.4774
Epoch 4/30
7/7 [==============================] - 100s 15s/step - loss: 0.5126 - accuracy: 0.7542 - val_loss: 1.2068 - val_accuracy: 0.5967
Epoch 5/30
7/7 [==============================] - 104s 15s/step - loss: 0.2860 - accuracy: 0.9072 - val_loss: 1.2871 - val_accuracy: 0.5885
Epoch 6/30
8/8 [==============================] - 14s 2s/step
Accuracy on Test Set: 0.4773662551440329
Macro Average F1 Score on Test Set: 0.3231197771587744
16/16 [==============================] - 20s 1s/step
Unique Predicted Labels: ['OBJ' 'SUBJ']


# With Attention

In [ ]:
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 10.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import f1_score

# Define Attention layer
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer

class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        et = K.squeeze(K.tanh(K.dot(x, self.W) + self.b), axis=-1)
        at = K.softmax(et)
        at = K.expand_dims(at, axis=-1)
        output = x * at
        return K.sum(output, axis=1)

# Load data
df_final = pd.read_csv("/content/CheckThat-Task2-Subjectivity/train_en.tsv", sep='\t')
df_test_final = pd.read_csv("/content/CheckThat-Task2-Subjectivity/dev_test_en.tsv", sep='\t')
df_submission_final = pd.read_csv("/content/CheckThat-Task2-Subjectivity/test_en.tsv", sep='\t')

# Tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_final['sentence'])

# Encode text data to sequences
X_train = tokenizer.texts_to_sequences(df_final['sentence'])
X_test = tokenizer.texts_to_sequences(df_test_final['sentence'])
X_submission = tokenizer.texts_to_sequences(df_submission_final['sentence'])

# Pad sequences to ensure uniform length
max_sequence_length = max([len(seq) for seq in X_train + X_test + X_submission])
X_train = pad_sequences(X_train, maxlen=max_sequence_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_sequence_length, padding='post')
X_submission = pad_sequences(X_submission, maxlen=max_sequence_length, padding='post')

# Encode labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df_final['label'])
y_test = label_encoder.transform(df_test_final['label'])

# Build LSTM model with Attention mechanism
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(units=512, return_sequences=True)))
model.add(Dropout(0.5))
model.add(AttentionLayer())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=30, batch_size=128, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Predict probabilities for test set
y_pred_probs = model.predict(X_test)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)

# Calculate accuracy
accuracy = (y_pred == y_test).mean()
print(f'Accuracy on Test Set: {accuracy}')

# Calculate macro average F1 score
f1 = f1_score(y_test, y_pred, average='macro')
print(f'Macro Average F1 Score on Test Set: {f1}')

# Predict probabilities for submission data
y_submission_pred_probs = model.predict(X_submission)

# Convert probabilities to class labels
y_submission_pred = np.argmax(y_submission_pred_probs, axis=1)
predicted_labels = label_encoder.inverse_transform(y_submission_pred)

# Check unique predicted labels
unique_labels = np.unique(predicted_labels)
print(f'Unique Predicted Labels: {unique_labels}')

# Create dataframe with sentence_id and predicted labels
df_submission_predicted = pd.DataFrame({'sentence_id': df_submission_final['sentence_id'], 'predicted_label': predicted_labels})

# Save dataframe to TSV file
df_submission_predicted.to_csv('submission_predictions_lstm_attention_custom.tsv', sep='\t', index=False)


Epoch 1/30
7/7 [==============================] - 50s 7s/step - loss: 0.6648 - accuracy: 0.6422 - val_loss: 0.7205 - val_accuracy: 0.4774
Epoch 2/30
7/7 [==============================] - 44s 6s/step - loss: 0.6625 - accuracy: 0.6410 - val_loss: 0.7325 - val_accuracy: 0.4774
Epoch 3/30
7/7 [==============================] - 43s 6s/step - loss: 0.6534 - accuracy: 0.6410 - val_loss: 0.7560 - val_accuracy: 0.4774
Epoch 4/30
8/8 [==============================] - 5s 468ms/step
Accuracy on Test Set: 0.4773662551440329
Macro Average F1 Score on Test Set: 0.3231197771587744
16/16 [==============================] - 9s 542ms/step
Unique Predicted Labels: ['OBJ']


In [ ]:
# Print the number of samples in each class
class_counts = df_final['label'].value_counts()
print(class_counts)


label
OBJ     532
SUBJ    298
Name: count, dtype: int64


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight

# Define Attention layer
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer

class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        et = K.squeeze(K.tanh(K.dot(x, self.W) + self.b), axis=-1)
        at = K.softmax(et)
        at = K.expand_dims(at, axis=-1)
        output = x * at
        return K.sum(output, axis=1)

# Load data
df_final = pd.read_csv("/content/CheckThat-Task2-Subjectivity/train_en.tsv", sep='\t')
df_test_final = pd.read_csv("/content/CheckThat-Task2-Subjectivity/dev_test_en.tsv", sep='\t')
df_submission_final = pd.read_csv("/content/CheckThat-Task2-Subjectivity/test_en.tsv", sep='\t')

# Tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_final['sentence'])

# Encode text data to sequences
X_train = tokenizer.texts_to_sequences(df_final['sentence'])
X_test = tokenizer.texts_to_sequences(df_test_final['sentence'])
X_submission = tokenizer.texts_to_sequences(df_submission_final['sentence'])

# Pad sequences to ensure uniform length
max_sequence_length = max([len(seq) for seq in X_train + X_test + X_submission])
X_train = pad_sequences(X_train, maxlen=max_sequence_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_sequence_length, padding='post')
X_submission = pad_sequences(X_submission, maxlen=max_sequence_length, padding='post')

# Encode labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df_final['label'])
y_test = label_encoder.transform(df_test_final['label'])

class_weights = compute_class_weight('balanced', classes=np.unique(df_final['label']), y=df_final['label'])

class_weights_dict = dict(enumerate(class_weights))

# Build more complex LSTM model with Attention mechanism
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(units=512, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(units=256, return_sequences=True)))
model.add(Dropout(0.5))
model.add(AttentionLayer())
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with class weights
model.fit(X_train, y_train, epochs=30, batch_size=128, validation_data=(X_test, y_test), callbacks=[early_stopping], class_weight=class_weights_dict)

# Predict probabilities for test set
y_pred_probs = model.predict(X_test)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)

# Calculate accuracy
accuracy = (y_pred == y_test).mean()
print(f'Accuracy on Test Set: {accuracy}')

# Calculate macro average F1 score
f1 = f1_score(y_test, y_pred, average='macro')
print(f'Macro Average F1 Score on Test Set: {f1}')

# Predict probabilities for submission data
y_submission_pred_probs = model.predict(X_submission)

# Convert probabilities to class labels
y_submission_pred = np.argmax(y_submission_pred_probs, axis=1)
predicted_labels = label_encoder.inverse_transform(y_submission_pred)

# Check unique predicted labels
unique_labels = np.unique(predicted_labels)
print(f'Unique Predicted Labels: {unique_labels}')

# Create dataframe with sentence_id and predicted labels
df_submission_predicted = pd.DataFrame({'sentence_id': df_submission_final['sentence_id'], 'predicted_label': predicted_labels})

# Save dataframe to TSV file
df_submission_predicted.to_csv('submission_predictions_lstm_attention_updated.tsv', sep='\t', index=False)


Epoch 1/30
7/7 [==============================] - 88s 13s/step - loss: 0.6979 - accuracy: 0.4614 - val_loss: 0.6926 - val_accuracy: 0.5226
Epoch 3/30
7/7 [==============================] - 88s 13s/step - loss: 0.6913 - accuracy: 0.5145 - val_loss: 0.6962 - val_accuracy: 0.4774
Epoch 4/30
7/7 [==============================] - 88s 13s/step - loss: 0.6959 - accuracy: 0.4880 - val_loss: 0.6920 - val_accuracy: 0.5021
Epoch 5/30
7/7 [==============================] - 87s 13s/step - loss: 0.6776 - accuracy: 0.5325 - val_loss: 0.7065 - val_accuracy: 0.5144
Epoch 6/30
7/7 [==============================] - 85s 12s/step - loss: 0.7749 - accuracy: 0.7325 - val_loss: 0.6983 - val_accuracy: 0.5267
Epoch 7/30
8/8 [==============================] - 10s 1s/step
Accuracy on Test Set: 0.5020576131687243
Macro Average F1 Score on Test Set: 0.34146341463414637
16/16 [==============================] - 16s 1s/step
Unique Predicted Labels: ['OBJ' 'SUBJ']


# Simple GRU


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import f1_score

# Load data
df_final = pd.read_csv("/content/CheckThat-Task2-Subjectivity/train_en.tsv", sep='\t')
df_test_final = pd.read_csv("/content/CheckThat-Task2-Subjectivity/dev_test_en.tsv", sep='\t')
df_submission_final = pd.read_csv("/content/CheckThat-Task2-Subjectivity/test_en.tsv", sep='\t')

# Tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_final['sentence'])

# Encode text data to sequences
X_train = tokenizer.texts_to_sequences(df_final['sentence'])
X_test = tokenizer.texts_to_sequences(df_test_final['sentence'])
X_submission = tokenizer.texts_to_sequences(df_submission_final['sentence'])

# Pad sequences to ensure uniform length
max_sequence_length = max([len(seq) for seq in X_train + X_test + X_submission])
X_train = pad_sequences(X_train, maxlen=max_sequence_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_sequence_length, padding='post')
X_submission = pad_sequences(X_submission, maxlen=max_sequence_length, padding='post')

# Encode labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df_final['label'])
y_test = label_encoder.transform(df_test_final['label'])

# Build a more complex GRU model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length))
model.add(Bidirectional(GRU(units=512, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(GRU(units=256, return_sequences=True)))
model.add(Dropout(0.5))
model.add(GRU(units=128))
model.add(Dropout(0.5))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=30, batch_size=128, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Predict probabilities for test set
y_pred_probs = model.predict(X_test)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)

# Calculate accuracy
accuracy = (y_pred == y_test).mean()
print(f'Accuracy on Test Set: {accuracy}')

# Calculate macro average F1 score
f1 = f1_score(y_test, y_pred, average='macro')
print(f'Macro Average F1 Score on Test Set: {f1}')

# Predict probabilities for submission data
y_submission_pred_probs = model.predict(X_submission)

# Convert probabilities to class labels
y_submission_pred = np.argmax(y_submission_pred_probs, axis=1)
predicted_labels = label_encoder.inverse_transform(y_submission_pred)

# Check unique predicted labels
unique_labels = np.unique(predicted_labels)
print(f'Unique Predicted Labels: {unique_labels}')

# Create dataframe with sentence_id and predicted labels
df_submission_predicted = pd.DataFrame({'sentence_id': df_submission_final['sentence_id'], 'predicted_label': predicted_labels})

# Save dataframe to TSV file
df_submission_predicted.to_csv('submission_predictions_gru_updated.tsv', sep='\t', index=False)


Epoch 1/30
7/7 [==============================] - 76s 9s/step - loss: 0.6729 - accuracy: 0.6000 - val_loss: 0.6960 - val_accuracy: 0.4774
Epoch 2/30
7/7 [==============================] - 61s 9s/step - loss: 0.6760 - accuracy: 0.6012 - val_loss: 0.7603 - val_accuracy: 0.4774
Epoch 3/30
7/7 [==============================] - 62s 9s/step - loss: 0.6615 - accuracy: 0.6337 - val_loss: 0.7236 - val_accuracy: 0.4774
Epoch 4/30
8/8 [==============================] - 8s 810ms/step
Accuracy on Test Set: 0.4773662551440329
Macro Average F1 Score on Test Set: 0.3231197771587744
16/16 [==============================] - 14s 888ms/step
Unique Predicted Labels: ['OBJ']
